In [1]:
#load in json file, named patents.json
from google.colab import files
uploaded = files.upload()

Saving patents.json to patents.json


In [2]:
import json

with open("patents.json") as f:
    patents = json.load(f)

In [7]:
pairs = []
for p in patents:
    title = p.get("title")
    desc = p.get("description")
    if title and desc:
        pairs.append((title, desc))

#print length of pairs to make sure they all loaded
len(pairs)



50

In [8]:
#creating our dataset based on the pairs
from torch.utils.data import Dataset

class PatentDataset(Dataset):
    def __init__(self, items):
        self.items = items

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        return self.items[idx]

dataset = PatentDataset(pairs)


In [9]:
#using small bert tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [10]:
#collate function to tokenize key-value (title-desc) pairs
def collate(batch):
    inputs, outputs = zip(*batch)
    x = tokenizer(list(inputs), padding=True, truncation=True, return_tensors="pt")
    y = tokenizer(list(outputs), padding=True, truncation=True, return_tensors="pt")
    return x["input_ids"], y["input_ids"]


In [11]:
from torch.utils.data import DataLoader

loader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate)

#will return number of sequences in the batch, followed by the max number of tokens a title used [2, x]
next(iter(loader))[0].shape

torch.Size([2, 6])

In [12]:
import torch
import torch.nn as nn

#creating a dumb neural network to test tokenization

class NeuralNetwork(nn.Module):
    def __init__(self, vocab, hidden=64):
        super().__init__()
        self.emb = nn.Embedding(vocab, hidden)
        self.fc = nn.Linear(hidden, vocab)

    def forward(self, ids):
        e = self.emb(ids)          # batch, seq, hidden
        h = e.mean(dim=1)          # batch, hidden
        return self.fc(h)          # batch, vocab

model = NeuralNetwork(tokenizer.vocab_size)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

In [13]:
#if this prints "loss: xx.xxx" then the data was tokenized successfully
for x, y in loader:
    logits = model(x)
    loss = loss_fn(logits, y[:, 0])
    loss.backward()
    opt.step()
    opt.zero_grad()
    print("loss:", loss.item())
    break


loss: 10.73243522644043
